In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import joblib
import json
import os
import pickle

In [6]:
# ==================== 1. ĐỌC DỮ LIỆU ====================
print("Đang đọc dữ liệu từ BIDMC CSV...")
df = pd.read_csv("data.csv")
df.columns = df.columns.str.strip()
print("Tên cột:", df.columns.tolist())

Đang đọc dữ liệu từ BIDMC CSV...
Tên cột: ['Time [s]', 'HR', 'PULSE', 'RESP', 'SpO2']


In [7]:
# ==================== 2. TẠO ĐẶC TRƯNG ====================
df['AvgBPM'] = df['PULSE']
df['A_total'] = df['RESP'] * 0.5
df = df.dropna(subset=['AvgBPM', 'A_total'])

print(f"\nĐã xử lý {len(df)} bản ghi")
print(df[['PULSE', 'RESP', 'AvgBPM', 'A_total']].head())


Đã xử lý 468 bản ghi
   PULSE  RESP  AvgBPM  A_total
0   93.0    25    93.0     12.5
1   93.0    25    93.0     12.5
2   93.0    25    93.0     12.5
3   93.0    26    93.0     13.0
4   93.0    26    93.0     13.0


In [8]:
# ==================== 3. TẠO NHÃN ====================
def classify_health_state(avg_bpm, a_total):
    if 60 <= avg_bpm <= 110:
        if a_total < 12: return 'Bình thường'
        elif a_total < 20: return 'Hoạt động nhẹ'
        elif a_total < 35: return 'Hoạt động trung bình'
        else: return 'Hoạt động mạnh'
    elif avg_bpm < 60:
        if a_total < 12: return 'Cảnh báo sức khỏe không ổn định'
        else: return 'Hoạt động mạnh bất thường'
    elif avg_bpm > 110:
        if a_total < 12: return 'Nhịp tim cao bất thường'
        elif a_total < 35: return 'Hoạt động thể chất mạnh'
        else: return 'Hoạt động cực độ'
    if avg_bpm < 50 and a_total < 12: return 'Sức khỏe siêu tốt'
    if avg_bpm > 150 and a_total > 35: return 'Cảnh báo nguy hiểm'
    return 'Không xác định'

df['label'] = df.apply(lambda row: classify_health_state(row['AvgBPM'], row['A_total']), axis=1)
df = df[df['label'] != 'Không xác định']

print("Số lớp:", df['label'].nunique())
print(df['label'].value_counts())

Số lớp: 2
label
Bình thường      427
Hoạt động nhẹ     41
Name: count, dtype: int64


In [9]:
# ==================== 4. CHUẨN BỊ DỮ LIỆU ====================
X = df[["AvgBPM", "A_total"]].values
y = df["label"].values

# Encode label
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Class weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

In [10]:
# ==================== 5. XÂY DỰNG MODEL NHẸ + CHỐNG OVERFIT ====================
model = keras.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(2,), kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(8, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
# Early stopping + Reduce LR
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7)
]

In [12]:
# Train
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=32,
    validation_data=(X_test, y_test),
    class_weight=class_weight_dict,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/200
12/12 [==============================] - 1s 18ms/step - loss: 0.7814 - accuracy: 0.2193 - val_loss: 0.8573 - val_accuracy: 0.0957 - lr: 0.0010
Epoch 2/200
12/12 [==============================] - 0s 5ms/step - loss: 0.7241 - accuracy: 0.3556 - val_loss: 0.8131 - val_accuracy: 0.4681 - lr: 0.0010
Epoch 3/200
12/12 [==============================] - 0s 6ms/step - loss: 0.7046 - accuracy: 0.4973 - val_loss: 0.7834 - val_accuracy: 0.6702 - lr: 0.0010
Epoch 4/200
12/12 [==============================] - 0s 6ms/step - loss: 0.6695 - accuracy: 0.5963 - val_loss: 0.7640 - val_accuracy: 0.8617 - lr: 0.0010
Epoch 5/200
12/12 [==============================] - 0s 4ms/step - loss: 0.6343 - accuracy: 0.7460 - val_loss: 0.7474 - val_accuracy: 0.8617 - lr: 0.0010
Epoch 6/200
12/12 [==============================] - 0s 5ms/step - loss: 0.5989 - accuracy: 0.7995 - val_loss: 0.7311 - val_accuracy: 0.8617 - lr: 0.0010
Epoch 7/200
12/12 [==============================] - 0s 7ms/step - loss: 0.

In [13]:
# ==================== 6. ĐÁNH GIÁ ====================
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nĐộ chính xác trên tập test: {test_acc:.4f}")

# In thêm thông tin
from sklearn.metrics import classification_report
y_pred = np.argmax(model.predict(X_test), axis=1)
print("\nBáo cáo phân loại:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Độ chính xác trên tập test: 0.9894
3/3 [==============================] - 0s 0s/step

Báo cáo phân loại:
               precision    recall  f1-score   support

  Bình thường       1.00      0.99      0.99        86
Hoạt động nhẹ       0.89      1.00      0.94         8

     accuracy                           0.99        94
    macro avg       0.94      0.99      0.97        94
 weighted avg       0.99      0.99      0.99        94



In [14]:
# ==================== 7. LƯU MODEL + SCALER + ENCODER (ĐÚNG CÁCH) ====================
model.save('health_model.h5')

# DÙNG 'wb' – QUAN TRỌNG!!!
with open('scaler_bidmc.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open('label_encoder_bidmc.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

print("ĐÃ LƯU MODEL + SCALER + ENCODER THÀNH CÔNG!")

ĐÃ LƯU MODEL + SCALER + ENCODER THÀNH CÔNG!


In [15]:
# ==================== 8. CONVERT SANG TENSORFLOW.JS ====================
import tensorflowjs as tfjs

print("\nĐang convert sang TensorFlow.js...")
tfjs.converters.save_keras_model(model, 'tfjs_model')

# Convert scaler + label → JSON
with open('scaler.json', 'w') as f:
    json.dump({'mean': scaler.mean_.tolist(), 'scale': scaler.scale_.tolist()}, f, indent=4)

with open('label_encoder.json', 'w') as f:
    json.dump({'classes': label_encoder.classes_.tolist()}, f, indent=4)

print("HOÀN TẤT! File TF.js + JSON đã tạo:")
print("   - tfjs_model/model.json + *.bin")
print("   - scaler.json")
print("   - label_encoder.json")


Đang convert sang TensorFlow.js...
HOÀN TẤT! File TF.js + JSON đã tạo:
   - tfjs_model/model.json + *.bin
   - scaler.json
   - label_encoder.json
